<h1> Extração de features de duas imagens com informações complementares e classificação por CNN </h1>

Primeiramente fazemos o alinhamento das imagens que foram obtidas por diferentes técnicas de imageamento. 
Neste exemplo utilizaram-se as obtidas por UV e IR.
Este é o código para alinhar as duas imagens por pontos-chaves (ORB):

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# ler duas imagens
img1 = cv.imread('uv20161004rosael-8_reduzida_para_257.jpg', cv.IMREAD_GRAYSCALE)
img2 = cv.imread('quiteriaIR_reduzida_para_257.jpg', cv.IMREAD_GRAYSCALE)

# Dar o ORB nelas
orb = cv.ORB_create()
kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)

# Dar o Brute Force Matcher e os descriptors bf.Match(des1, des2)
bf = cv.BFMatcher (cv.NORM_HAMMING, crossCheck = True)
matches = bf.match(des1, des2)

# Ordena-os por distância
matches = sorted(matches, key = lambda x:x.distance)

# Cria variáveis com uma das dimensões do tamanho do número de matches 
matches = matches[:int(len(matches)*90)]
no_of_matches = len(matches)
p1 = np.zeros((no_of_matches, 2))
p2 = np.zeros((no_of_matches, 2))

# Atribui o queryIdx dos kp1 dos matches da img1 à variável p1 e os trainIdx da img2 (à variável p2)
for i in range(len(matches)):
    p1[i, :] = kp1[matches[i].queryIdx].pt
    p2[i, :] = kp2[matches[i].trainIdx].pt
    
# Cria máscara, homografia e imagem transformada por elas
homography, mask = cv.findHomography(p1, p2, cv.RANSAC)
transformed_img = cv.warpPerspective(img1, homography, (706,705))
cv.imwrite('imagem_transformada.jpg', transformed_img)
                                     
# Desenha os 10 primeiros matches
img3 = cv.drawMatches(img1,kp1,img2,kp2,matches[:10],None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img3),plt.show()

Após alinhadas as imagens, sabemos que os pixelsets de uma correspondem à mesma região de pintura dos pixelsets da outra. Assim, podemos extrair informações de cada pixelset das duas imagens somadas, ou melhor, de trechos de um array que é a soma das duas obtido através de:

In [ ]:
import cv2 as cv
import numpy as np

path = 'original'

img1 = cv.imread (path + '/' + 'MariaQuiteria-DomenicoFailutti-MP-USP-RAD-WEB-PedroCampos-MarciaRizzutto-IFUSP-2_reduzida_para_257_margem.jpg')
img1array = np.array(img1)
img2 = cv.imread (path + '/' + 'quiteriaIR_reduzida_para_257.jpg')
img2array = np.array(img2)

imgs_somadas = ((257,257,6))

imgs_somadas [:,:,0:3] = img1array
imgs_somadas [:,:,3:] = img2array

np.save('imgs_somadas.npy',imgs_somadas)

Para extrair as features dos pixelsets, vamos recortar o array em que já estão as infos somadas de cada imagem. Os pixelsets serão os dados que informaremos à CNN para ela aprender quais se tratam de intervenção e quais não, conforme a pasta de onde eles são lidos (conforme os labels):

In [ ]:
import numpy as np
import cv2 as cv
import pandas as pd
import os
from tqdm import tqdm
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from sklearn.model_selection import train_test_split

def treina_modelo(modelo, num_classes):
    model.add (Conv2D(32,(3,3),
                #activation = 'relu',
                input_shape = (5,5,3)))
                #padding = 'same'))
    model.add(MaxPooling2D(pool_size=(32,32),
                           padding='same'))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(2,activation='softmax'))
    print ('rodando treinamento do modelo')
    print (type(model))
    model.compile(loss=keras.losses.mean_squared_error,
                  optimizer = 'adam',
                  metrics = ['accuracy'])
    return model

def le_imagens(path):
    x = []
    y = []

    labels = os.listdir(path)
    print('-'*10)
    print(labels)
    print('-'*10)
    
    for label in labels:
        print('Loading: ',label)
        files = os.listdir(path + '/' + label)
        print('-'*10)
        print(files)
        print('-'*10)
        for file in tqdm(files):
            img = cv.imread (path + '/' + label + '/' + file)
            imgarray = np.array(img)
            x.append(imgarray)
            y.append(label)
    x = np.array(x)
    y = np.array(y)
    y = pd.get_dummies(y).values
    
    return x, y, img

if __name__=="__main__":
    path = r'imagens_recortadas1'
    num_classes = 2
    epochs = 10
    batch_size = 36
    x, y, imagem = le_imagens(path) 
    print (type(x))
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.33, random_state = 23)
    x_test, x_val, y_test, y_val = train_test_split(x_val, y_val, test_size = 0.33, random_state = 23)
    
    model = tf.keras.Sequential()

    model = treina_modelo(model, num_classes)

    model.fit(x_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        validation_data=(x_val, y_val))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    model.save("portrait_net_model.h5")
    model.summary()

Após aplicar essas convoluções, procede-se à aplicação da Principal Component Analysis (PCA) para observar como as informações estão: ->>> É aqui que preciso de ajuda! <<<-

In [ ]:
from sklearn.decomposition import PCA

df = pd.DataFrame(dados_saida_do_modelo)
X = df.drop('target',1) # corrigir para retirar a ultima coluna, simplesmente
    y = df['target'] # corrigir para pegar a ultima coluna, não 'target'
    pca = PCA(n_components=2)
    pca.fit(X)

    mean_vec = np.mean(X, axis=0)
    M = X - mean_vec
    C = M.T.dot(M) / (X.shape[0]-1)
    autovalores, autovetores = np.linalg.eig(C)
    print (autovalores)
    
    pares_de_autos = [
        (
                np.abs(autovalores[i]),
                autovetores[:,i]
        ) for i in range(len(autovalores)) 
    ]
    pares_de_autos.reverse()
    
   #graficando
    n_componentes = 2
    autovetores = [p[1] for p in pares_de_autos]
    A = autovetores[0:n_componentes]
    X = np.dot(X,np.array(A).T)
    
    new_df = pd.DataFrame(X,columns=['pc1','pc2'])
    new_df['target']=df['target']
    sns.pairplot(
    	new_df, vars = ['pc1','pc2'], hue='target', diag_kind="hist"
    )
    plt.show() 

Para extrair os dados a serem utilizados na PCA, ao invés de utilizar o modelo Sequential do código acima, também experimentou-se a seguinte extração por Transfer Learning da VGG16:

(a modificar ainda, para ler a soma das imagens, não somente a de Raio X)

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights='imagenet', include_top=False)

x = []

img_path = 'original/MariaQuiteria-DomenicoFailutti-MP-USP-RAD-WEB-PedroCampos-MarciaRizzutto-IFUSP-2_reduzida_para_257.jpg'
img = image.load_img(img_path,target_size=(224,224))            
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)
print(features.shape)
print(features)

In [ ]:
(inserir resultado da PCA)

Uma alternativa para esse processamento acima é o Autoencoder, em que não utilizaríamos os labels "original" ou "com_intervencao" para treinar o sistema, mas sim realizaríamos um aprendizado não-supervisionado:

In [ ]:
import os
import cv2 as cv
import numpy as np
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from sklearn.tree import DecisionTreeClassifier

# modela o codificador e o decodificador
def build_model(x,y):
    
    #1a camada de convolução
    model.add(Conv2D(16, (3, 3), padding='same', input_shape=(224,224,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    
    #2a camada de convolução
    model.add(Conv2D(2,(3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    #-------------------------
    #3a camada de convolução
    model.add(Conv2D(2,(3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(UpSampling2D((2, 2)))
    
    #4a camada de convolução
    model.add(Conv2D(16,(3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(UpSampling2D((2, 2)))
    
    #-------------------------
    
    model.add(Conv2D(3,(3, 3), padding='same'))
    model.add(Activation('sigmoid'))
    
    model.summary()
    
    # Compila o modelo
    model.compile(optimizer='adadelta', loss='binary_crossentropy')
    
    return model

# le dados
def read_data(path):
    x = []
 
    files = os.listdir(path)
    for file in tqdm(files):
        img = cv.imread (path + '/' + file)
        imgarray = np.array(img)
        x.append(imgarray)
       
    x = np.array(x)
    
    return x

# main
if __name__=="__main__":
    
    path = r'array_selecionado_para_autoencoder'
    x = read_data(path)
    y = x[0] # trocar por um Flatten do x

    batch_size = 32
    epochs = 60
    model = Sequential()
    model = build_model(x,y) #ver se esta linha está funcionando mesmo

    model.save("autoencoder_model.h5")

# ------>>>>>>>>>>>>>> Árvore de decisão:
    clf = DecisionTreeClassifier(model)

Após extrair as features, pretende-se aplicar ao modelo uma camada de Árvore de Decisão, para dividir em duas 
classes os recortes que foram processados: Original e Com intervenção.